In [2]:
%logstop
%logstart -rtq ~/.logs/nlp.py append
%matplotlib inline 
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

In [3]:
from static_grader import grader

In [3]:
from IPython.display import clear_output

import warnings
warnings.filterwarnings('ignore')

# NLP Miniproject

## Introduction

The objective of this miniproject is to gain experience with natural language processing and how to use text data to train a machine learning model to make predictions. For the miniproject, we will be working with product review text from Amazon. The reviews are for only products in the "Electronics" category. The objective is to train a model to predict the rating, ranging from 1 to 5 stars.

## Scoring

For most of the questions, you will be asked to submit the `predict` method of your trained model to the grader. The grader will use the passed `predict` method to evaluate how your model performs on a test set with respect to a reference model. The grader uses the [R<sup>2</sup>-score](https://scikit-learn.org/stable/modules/model_evaluation.html#r2-score) for model evaluation. If your model performs better than the reference solution, then you can score higher than 1.0. For the last question, you will submit the results of an analysis and your passed answer will be compared directly to the reference solution.

## Downloading and loading the data

The data set is available on Amazon S3 and comes as a compressed file where each line is a JSON object. To load the data set, we will need to use the `gzip` library to open the file and decode each JSON into a Python dictionary. In the end, we have a list of dictionaries, where each dictionary represents an observation.

In [4]:
%%bash
mkdir data
wget http://dataincubator-wqu.s3.amazonaws.com/mldata/amazon_electronics_reviews_training.json.gz -nc -P ./data

mkdir: cannot create directory ‘data’: File exists
File ‘./data/amazon_electronics_reviews_training.json.gz’ already there; not retrieving.



In [5]:
import gzip
import json as json

with gzip.open("data/amazon_electronics_reviews_training.json.gz", "r") as f:                                  
    data = [json.loads(line) for line in f]

In [6]:
data[0]

{'reviewerID': 'A238V1XTSK9NFE',
 'asin': 'B00004VX3T',
 'reviewerName': 'Andrew Lynn',
 'helpful': [2, 2],
 'reviewText': "I bought this mouse to use with my laptop because I don't like those little touchpads.  I could not be happier.Since it's USB, I can plug it in with the computer already on and expect it to work automatically.  Since it's optical (the new kind, not to be confused with the old Sun optical mice that required a special checkered mouse pad) it works on most surfaces, including my pant legs, my couch, and random tables that I put my laptop down on.  It's also light and durable, features that help with portability.The wheel is surprisingly useful.  In addition to scrolling, it controls zoom and pan in programs like Autocad and 3D Studio Max.  I can no longer bear using either of these programs without it.One complaint - the software included with the Internet navigation features is useless.  Don't bother installing it if you have a newer Windows version that automatical

The ratings are stored in the keyword `"overall"`. You should create an array of the ratings for each review, preferably using list comprehensions.

In [11]:
X=data[0]
X['overall']

5.0

In [12]:
ratings = [X['overall'] for X in data]

In [13]:
ratings[:10]

[5.0, 1.0, 4.0, 5.0, 3.0, 5.0, 3.0, 5.0, 4.0, 5.0]

**Note**, the test set used by the grader is in the same format as that of `data`, a list of dictionaries. Your trained model needs to accept data in the same format. Thus, you should use `Pipeline` when constructing your model so that all necessary transformation needed are encapsulated into a single estimator object.

## Question 1: Bag of words model

Construct a machine learning model trained on word counts using the bag of words algorithm. Remember, the bag of words is implemented with `CountVectorizer`. Some things you should consider:

* The reference solution uses a linear model and you should as well; use either `Ridge` or `SGDRegressor`.
* The text review is stored in the key `"reviewText"`. You will need to construct a custom transformer to extract out the value of this key. It will be the first step in your pipeline.
* Consider what hyperparameters you will need to tune for your model.
* Subsampling the training data will boost training times, which will be helpful when determining the best hyperparameters to use. Note, your final model will perform best if it is trained on the full data set.
* Including stop words may help with performance.

In [12]:
reviewText = [text['reviewText'] for text in data]

In [13]:
reviewText[1]

'One by one, all of the discs went bad within a 6 months period. It was a real pain. As a result, I would tend to not buy Memorex discs again.'

In [15]:
from sklearn.base import BaseEstimator, TransformerMixin

#class keyTransformer(BaseEstimator, TransformerMixin):
class keySelector(BaseEstimator, TransformerMixin):
 
    #class keyTransformer
    def __init__(self, key):
        self.key = key
        
    def fit(self, X, y = None):
        return self
    
    def transform(self, X):
        return [text[self.key] for text in X]

In [16]:
key = keySelector("reviewText")
key_trs = key.fit_transform(data)
print(key_trs[-1])

I only use the remote control twice a year - at Christmas to take group photos of my family and my wife's family - but this little gadget is worth every penny. I used to set the timer, then start it running while I ran around to get in the picture. It's so much easier to trigger the shutter by remote control. I've used it with a D200 and D300, and it works perfectly.


In [16]:
#import spacy 

#nlp = spacy.load('en')

In [17]:
#from spacy.lang.en import STOP_WORDS

In [18]:
#stop_words_str = " ".join(STOP_WORDS)

In [19]:
#stop_words_lemma = set(word.lemma_ for word in nlp(stop_words_str))

In [19]:
#from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer #, TfidfTransformer
from sklearn.pipeline import Pipeline
#from sklearn.linear_model import Ridge
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer #TfidVectorizer
from sklearn.linear_model import Ridge
#, SGDRegressor
#from spacy.lang.en import STOP_WORDS

In [21]:
vectorizer = CountVectorizer()

bag_of_words_model = Pipeline([
    ('selector', keySelector('reviewText')),
    ('vectorizer', HashingVectorizer()),
    ('regressor', Ridge(alpha=0.6))
])

bag_of_words_model.fit(data, ratings);

In [22]:
grader.score.nlp__bag_of_words_model(bag_of_words_model.predict)

Your score: 1.835


## Question 2: Normalized model

Using raw counts will not be as effective compared if we had normalized the counts. There are several ways to normalize raw counts; the `HashingVectorizer` class has the keyword `norm` and there is also the `TfidfTransformer` and `TfidfVectorizer` that perform tf-idf weighting on the counts. Apply normalization to your model to improve performance.

In [20]:

normalized_model = Pipeline([
    ('selector', KeySelector('reviewText')),
    ('vectorizer', HashingVectorizer()),
    ('regressor', Ridge())
])

normalized_model.fit(data, ratings)

Pipeline(steps=[('selector', KeySelector(key='reviewText')),
                ('vectorizer', HashingVectorizer()), ('regressor', Ridge())])

In [21]:
grader.score.nlp__normalized_model(normalized_model.predict)

Your score: 1.034


## Question 3: Bigrams model

The model performance may increase when including additional features generated by counting bigrams. Include bigrams to your model. When using more features, the risk of overfitting increases. Make sure you try to minimize overfitting as much as possible.

In [22]:

bigrams_model = Pipeline([
    ('selector', KeySelector('reviewText')),
    ('vectorizer', HashingVectorizer(lowercase=True, ngram_range=(1,2))),
    ('regressor', Ridge())
])

bigrams_model.fit(data, ratings)

Pipeline(steps=[('selector', KeySelector(key='reviewText')),
                ('vectorizer', HashingVectorizer(ngram_range=(1, 2))),
                ('regressor', Ridge())])

In [23]:
grader.score.nlp__bigrams_model(bigrams_model.predict)

Your score: 1.152


## Question 4: Polarity analysis

Let's derive some insight from our analysis. We want to determine the most polarizing words in the corpus of reviews. In other words, we want identify words that strongly signal a review is either positive or negative. For example, we understand a word like "terrible" will mostly appear in negative rather than positive reviews. The naive Bayes model calculates probabilities such as $P(\text{terrible } | \text{ negative})$, the probability the word "terrible" appears in the text, given that the review is negative. Using these probabilities, we can derive a **polarity score** for each counted word,

$$
\text{polarity} =  \log\left(\frac{P(\text{word } | \text{ positive})}{P(\text{word } | \text{ negative})}\right).
$$ 

The polarity analysis is an example where a simpler model offers more explicability than a more complicated model. For this question, you are asked to determine the top thirty words with the largest positive **and** largest negative polarity, for a total of sixty words. For this analysis, you should:

1. Use the naive Bayes model, `MultinomialNB`.
1. Use tf-idf weighting.
1. Remove stop words.

A trained naive Bayes model stores the log of the probabilities in the attribute `feature_log_prob_`. It is a NumPy array of shape (number of classes, the number of features). You will need the mapping between feature index to word. For this problem, you will use a different data set; it has been processed to only include reviews with one and five stars. You can download it below.

In [24]:
%%bash
wget http://dataincubator-wqu.s3.amazonaws.com/mldata/amazon_one_and_five_star_reviews.json.gz -nc -P ./data

--2022-02-16 09:32:23--  http://dataincubator-wqu.s3.amazonaws.com/mldata/amazon_one_and_five_star_reviews.json.gz
Resolving dataincubator-wqu.s3.amazonaws.com (dataincubator-wqu.s3.amazonaws.com)... 54.231.135.177
Connecting to dataincubator-wqu.s3.amazonaws.com (dataincubator-wqu.s3.amazonaws.com)|54.231.135.177|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2970853 (2.8M) [application/x-gzip]
Saving to: ‘./data/amazon_one_and_five_star_reviews.json.gz’

     0K .......... .......... .......... .......... ..........  1% 3.88M 1s
    50K .......... .......... .......... .......... ..........  3% 7.78M 1s
   100K .......... .......... .......... .......... ..........  5% 33.3M 0s
   150K .......... .......... .......... .......... ..........  6% 61.3M 0s
   200K .......... .......... .......... .......... ..........  8% 9.03M 0s
   250K .......... .......... .......... .......... .......... 10% 80.3M 0s
   300K .......... .......... .......... .......... .....

In order to avoid memory issues, let's delete the older data.

In [25]:
del data, ratings

In [26]:
import numpy as np
from sklearn.naive_bayes import MultinomialNB

with gzip.open("data/amazon_one_and_five_star_reviews.json.gz", "r") as f:
    data_polarity = [json.loads(line) for line in f]

ratings = [review['overall'] for review in data_polarity]


In [28]:
pipe = Pipeline([
    ("selector", KeySelector("reviewText")),
    ("vectorizer", TfidfVectorizer(stop_words = STOP_WORDS)),
    ("classifier", MultinomialNB())
])

pipe.fit(data_polarity, ratings)

Pipeline(steps=[('selector', KeySelector(key='reviewText')),
                ('vectorizer',
                 TfidfVectorizer(stop_words={"'d", "'ll", "'m", "'re", "'s",
                                             "'ve", 'a', 'about', 'above',
                                             'across', 'after', 'afterwards',
                                             'again', 'against', 'all',
                                             'almost', 'alone', 'along',
                                             'already', 'also', 'although',
                                             'always', 'am', 'among', 'amongst',
                                             'amount', 'an', 'and', 'another',
                                             'any', ...})),
                ('classifier', MultinomialNB())])

In [33]:
#retrieve features' log probabilities
log_prob = pipe['classifier'].feature_log_prob_

#compute polarity
polarity = log_prob[0,:] - log_prob[1,:]

#get feature names
terms = pipe['vectorizer'].get_feature_names()

#assign each feature to corresponding polarity
terms_polarity = list(zip(polarity, terms))

#sort features_polarity elements with respect to polarity
sorted_terms_polarity = sorted(terms_polarity)

#select N highest polirized terms
N = 60
highest_polarized_terms = sorted_terms_polarity[:N//2] + sorted_terms_polarity[-N//2:]


#extract the terms with highest polarization
top_60 = [term for polarity, term in highest_polarized_terms]

In [34]:
grader.score.nlp__most_polar(top_60)

Your score: 1.000


## Question 5: Topic modeling [optional]

Topic modeling is the analysis of determining the key topics or themes in a corpus. With respect to machine learning, topic modeling is an unsupervised technique. One way to uncover the main topics in a corpus is to use [non-negative matrix factorization](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.NMF.html). For this question, use non-negative matrix factorization to determine the top ten words for the first twenty topics. You should submit your answer as a list of lists. What topics exist in the reviews?

In [35]:
from sklearn.decomposition import NMF
 

*Copyright &copy; 2022 WorldQuant University. This content is licensed solely for personal use. Redistribution or publication of this material is strictly prohibited.*